In [1]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output
import pandas as pd
import tabulate

r.seed(14)

options = {}

# Uncomment if you are using a WLS Gurobi license
# and write your credentials in credentials.txt
with open('credentials.txt') as f: 
    data = f.read() 
options = json.loads(data)

In [2]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 100,
                      min_setup_time = 1,max_setup_time = 100):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 4
M_cardinality = 2

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality,
                                   min_execution_time = 1, max_execution_time = 5,
                                   min_setup_time = 1, max_setup_time = 5)

# Tabulate P_dict
P_list = list(P_dict.items())
print(tabulate.tabulate(P_list, headers=['(i, j)', 'Execution Time']))
# Tabulate S_dict
S_list = list(S_dict.items())
print(tabulate.tabulate(S_list, headers=['(i, j, k)', 'Setup Time']))


(i, j)      Execution Time
--------  ----------------
(1, 1)                   1
(1, 2)                   5
(1, 3)                   5
(1, 4)                   2
(2, 1)                   3
(2, 2)                   3
(2, 3)                   3
(2, 4)                   1
(i, j, k)      Setup Time
-----------  ------------
(1, 1, 1)               4
(1, 1, 2)               3
(1, 1, 3)               4
(1, 1, 4)               4
(1, 2, 1)               4
(1, 2, 2)               1
(1, 2, 3)               3
(1, 2, 4)               2
(1, 3, 1)               3
(1, 3, 2)               3
(1, 3, 3)               3
(1, 3, 4)               3
(1, 4, 1)               5
(1, 4, 2)               2
(1, 4, 3)               2
(1, 4, 4)               5
(2, 1, 1)               3
(2, 1, 2)               2
(2, 1, 3)               1
(2, 1, 4)               1
(2, 2, 1)               1
(2, 2, 2)               5
(2, 2, 3)               3
(2, 2, 4)               1
(2, 3, 1)               1
(2, 3, 2)               3
(2

In [3]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

In [4]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)
# clear_output(wait=True)
for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Job 1 completed at 1.0s
Job 2 completed at 3.0s
Job 3 completed at 9.0s
Job 4 completed at 7.0s
Job 1 has been assigned to machine 1
Job 4 has been assigned to machine 1
Job 2 has been assigned to machine 2
Job 3 has been assigned to machine 2
Job 0 scheduled in sequence of job 1 in machine 1
Job 1 scheduled in sequence of job 4 in machine 1
Job 4 scheduled in sequence of job 0 in machine 1
Job 0 scheduled in sequence of job 2 in machine 2
Job 2 scheduled in sequence of job 3 in machine 2
Job 3 scheduled in sequence of job 0 in machine 2
The makespan has been minimized to 9.0s


In [5]:
s = MPA(execution_times = P_dict, setup_times = S_dict, t_max=10800) #  max_time = 1800
decision_variables,maximum_makespan = s.solve(options=options)
for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Iteration: 1
Best makespan: inf
Best solution:
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Master solution:
(1, 0, 4) = 1.0
(1, 3, 0) = 1.0
(1, 4, 3) = 1.0
(2, 0, 2) = 1.0
(2, 1, 0) = 1.0
(2, 2, 1) = 1.0
Master assignments:
(1, 3) = 1
(1, 4) = 1
(2, 1) = 1
(2, 2) = 1
Master makespan: 7.0
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2519400
Academic license 2519400 - for non-commercial use only - registered to gi___@studenti.units.it
Sequence solution:
(1, 0, 4)
(1, 3, 0)
(1, 4, 3)
(2, 0, 2)
(2, 1, 0)
(2, 2, 1)
Sequence makespan: 9
C_max: {1: 9, 2: 7}
theta: {(1, 1): 5, (1, 2): 8, (1, 3): 8, (1, 4): 7, (2, 1): 6, (2, 2): 8, (2, 3): 6, (2, 4): 6}
N_h: {1: [3, 4], 2: [1, 2]}
Iteration: 2
Best makespan: 9
Best solution:
(1, 0, 4) = 1
(1, 3, 0) = 1
(1, 4, 3) = 1
(2, 0, 2) = 1
(2, 1, 0) = 1
(2, 2, 1) = 1
Set parame